# Setup

In [43]:
# --- 1) Setup ---
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --- 2) Load data ---
df = pd.read_csv(
    "/Users/amelijaancupane/Desktop/TU_DELFT/Year 2/Advanced Data Analysis/NS_Project/Group2_NS-1/data/complete_dataset_clean_vf.csv",
    sep=";"
)

# --- 3) Data cleaning---

# drop rows with missing target
df = df.dropna(subset=["REALISATIE"]).reset_index(drop=True)
df["REALISATIE"] = df["REALISATIE"].astype(float)


# booleans to ints (safer for sklearn)
for col in ["Cancelled", "ExtraTrain", "Previous train canceled", "Previous train delayed"]:
    if col in df.columns:
        df[col] = df[col].astype(int)


# --- 4) Target and features ---
y = df["REALISATIE"]

base_features = ["DAGDEELTREIN", "train_type", "PROGNOSE_REIZEN"]

weather_features = [c for c in [
    "Rain_flag","Heavy_Rain_flag","Gusts_flag","Storms_flag","Warm_flag","Cold_flag","Sunny_flag"
] if c in df.columns]

disrupt_features = [c for c in [
    "Cancelled","ExtraTrain","delay_category","disruption_category",
    "Previous train canceled","Previous train delayed", "disrupt_any"
] if c in df.columns]


features_all = [c for c in (base_features + weather_features + disrupt_features) if c in df.columns]

features_all


['DAGDEELTREIN',
 'train_type',
 'PROGNOSE_REIZEN',
 'Rain_flag',
 'Heavy_Rain_flag',
 'Gusts_flag',
 'Storms_flag',
 'Warm_flag',
 'Cold_flag',
 'Sunny_flag',
 'Cancelled',
 'ExtraTrain',
 'delay_category',
 'disruption_category',
 'Previous train canceled',
 'Previous train delayed',
 'disrupt_any']

## Linear Regression

In [45]:
X = df[features_all].copy()

# one-hot encode categoricals
categorical_cols = [c for c in ["DAGDEELTREIN", "train_type", "delay_category", "disruption_category"] if c in X.columns]
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print(f"Total model features (post one-hot): {X.shape[1]}")

# --- 5) Chronological split: last 7 days as TEST ---
cutoff_day = df["DAGNR"].max() - 7
train_mask = df["DAGNR"] <= cutoff_day
test_mask  = df["DAGNR"] >  cutoff_day

X_train = X[train_mask]
y_train = y[train_mask]
X_test  = X[test_mask]
y_test  = y[test_mask]

print("Train:", X_train.shape, "| Test:", X_test.shape)

# --- 6) Train & evaluate Linear Regression on full feature set ---
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2   = r2_score(y_test, y_pred)

print("\nLinear Regression — Test results (full feature set):")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

# Operator baseline on the same TEST split
op_test = df.loc[test_mask, "PROGNOSE_REIZEN"].astype(float)
op_mae  = mean_absolute_error(y_test, op_test)
op_rmse = np.sqrt(mean_squared_error(y_test, op_test))
op_r2   = r2_score(y_test, op_test)

print("\nOperator Forecast (PROGNOSE_REIZEN) — Test results:")
print(f"MAE: {op_mae:.2f}")
print(f"RMSE: {op_rmse:.2f}")
print(f"R²: {op_r2:.3f}")

# --- 7) Compare multiple feature subsets (aligned with trees-style features) ---
feature_sets = {
    "Set 1: Base + Delay cats": base_features + ["delay_category"],
    "Set 2: Base + Disruption cats": base_features + ["disruption_category"],
    "Set 3: Base + Weather": base_features + weather_features,
    "Set 4: Base + Disruptions full": base_features + disrupt_features,
    "Set 5: Base + Delay and Disruption cats": base_features + ["delay_category", "disruption_category"],
    "Set 6: ALL": features_all,  # 
    
}

# === Build comparison table (incl. disrupted metrics) & save the best overall-MAE model ===

# Consistent one-hot categories
FORCE_CAT = {"DAGDEELTREIN", "train_type", "delay_category", "disruption_category"}

# Disrupted mask (OR rule)
NO_DELAY_LABEL = "no delay (0-1 min)"
NO_DISR_LABEL  = "no disruption"
def disrupted_mask(idx):
    dcat = df.loc[idx, "delay_category"].astype(str).str.strip().str.lower() if "delay_category" in df.columns else "nan"
    disc = df.loc[idx, "disruption_category"].astype(str).str.strip().str.lower() if "disruption_category" in df.columns else "nan"
    return (dcat != NO_DELAY_LABEL) | (disc != NO_DISR_LABEL)

test_idx = df.index[test_mask]
dis_mask_test = disrupted_mask(test_idx).values

results = []
best_overall = {"mae": np.inf, "name": None, "model": None, "cols": None}

for name, feat_list in feature_sets.items():
    # keep only existing cols
    feat_list = [c for c in feat_list if c in df.columns]
    X_sub = df[feat_list].copy()

    # one-hot: union of forced cats + auto object/string cols
    auto_cat = [c for c in feat_list if X_sub[c].dtype == "object" or pd.api.types.is_string_dtype(X_sub[c])]
    cat_cols = sorted(set(auto_cat) | (set(feat_list) & FORCE_CAT))
    X_sub = pd.get_dummies(X_sub, columns=cat_cols, drop_first=True)

    # split
    X_tr = X_sub[train_mask]
    y_tr = y[train_mask]
    X_te = X_sub[test_mask]
    y_te = y[test_mask]

    # fit plain Linear for fair comparison
    m = LinearRegression()
    m.fit(X_tr, y_tr)
    y_hat = m.predict(X_te)

    # overall metrics
    mae  = mean_absolute_error(y_te, y_hat)
    rmse = np.sqrt(mean_squared_error(y_te, y_hat))
    r2   = r2_score(y_te, y_hat)

    # disrupted-only metrics
    if dis_mask_test.any():
        mae_d  = mean_absolute_error(y_te[dis_mask_test], y_hat[dis_mask_test])
        rmse_d = np.sqrt(mean_squared_error(y_te[dis_mask_test], y_hat[dis_mask_test]))
        r2_d   = r2_score(y_te[dis_mask_test], y_hat[dis_mask_test])
    else:
        mae_d = rmse_d = r2_d = np.nan

    results.append({
        "Feature Set": name,
        "MAE (Model)": round(mae, 2),
        "RMSE (Model)": round(rmse, 2),
        "R² (Model)": round(r2, 3),
        "MAE (Disrupted)": round(mae_d, 2),
        "RMSE (Disrupted)": round(rmse_d, 2),
        "R² (Disrupted)": round(r2_d, 3),
        "n_features": X_tr.shape[1]
    })

    # keep best by overall MAE
    if mae < best_overall["mae"]:
        best_overall = {"mae": mae, "name": name, "model": m, "cols": X_sub.columns.tolist()}

# Operator baseline row (overall + disrupted)
op_test = df.loc[test_mask, "PROGNOSE_REIZEN"].astype(float).values
y_test_all = y_test.values
op_mae  = round(mean_absolute_error(y_test_all, op_test), 2)
op_rmse = round(np.sqrt(mean_squared_error(y_test_all, op_test)), 2)
op_r2   = round(r2_score(y_test_all, op_test), 3)

if dis_mask_test.any():
    op_mae_d  = round(mean_absolute_error(y_test_all[dis_mask_test], op_test[dis_mask_test]), 2)
    op_rmse_d = round(np.sqrt(mean_squared_error(y_test_all[dis_mask_test], op_test[dis_mask_test])), 2)
    op_r2_d   = round(r2_score(y_test_all[dis_mask_test], op_test[dis_mask_test]), 3)
else:
    op_mae_d = op_rmse_d = op_r2_d = np.nan

operator_row = {
    "Feature Set": "Operator Forecast (baseline)",
    "MAE (Model)": op_mae,
    "RMSE (Model)": op_rmse,
    "R² (Model)": op_r2,
    "MAE (Disrupted)": op_mae_d,
    "RMSE (Disrupted)": op_rmse_d,
    "R² (Disrupted)": op_r2_d,
    "n_features": 1
}

results_df = pd.DataFrame([operator_row] + results).sort_values(by="MAE (Model)").reset_index(drop=True)

# pretty print table
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", 220)
print("\n=== Comparison of feature sets (Linear Regression) ===")
print(results_df)

# Save THE model & context for later cells
THE_FEATURE_SET_NAME = best_overall["name"]
THE_FEATURE_COLUMNS  = best_overall["cols"]      # post-one-hot columns
THE_MODEL_PIPE       = best_overall["model"]     # already fit on train split columns above

print(f"\nSaved THE model: {THE_FEATURE_SET_NAME}  | Overall Test MAE={best_overall['mae']:.2f}  | n_features={len(THE_FEATURE_COLUMNS)}")



Total model features (post one-hot): 26
Train: (75158, 26) | Test: (23459, 26)

Linear Regression — Test results (full feature set):
MAE: 63.21
RMSE: 92.14
R²: 0.817

Operator Forecast (PROGNOSE_REIZEN) — Test results:
MAE: 61.69
RMSE: 99.62
R²: 0.786

=== Comparison of feature sets (Linear Regression) ===
                               Feature Set  MAE (Model)  RMSE (Model)  R² (Model)  MAE (Disrupted)  RMSE (Disrupted)  R² (Disrupted)  n_features
0           Set 4: Base + Disruptions full        59.21         91.04       0.821            67.84            104.50           0.801          19
1  Set 5: Base + Delay and Disruption cats        59.45         92.93       0.814            69.05            108.22           0.786          14
2                 Set 1: Base + Delay cats        59.50         92.99       0.814            69.19            108.35           0.786          11
3            Set 2: Base + Disruption cats        59.68         93.59       0.811            69.80            11

### Tuning the LR Model

In [46]:
# Tune THE model's feature set (fix: rebuild from raw features, reapply one-hot, align columns)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# --- 1) Rebuild X from the RAW features of THE_FEATURE_SET_NAME, re-encode exactly like in Cell 1 ---
raw_feat_list = feature_sets[THE_FEATURE_SET_NAME]                   # raw columns (pre one-hot)
raw_feat_list = [c for c in raw_feat_list if c in df.columns]        # keep existing

X_raw = df[raw_feat_list].copy()

FORCE_CAT = {"DAGDEELTREIN","train_type","delay_category","disruption_category"}
auto_cat = [c for c in raw_feat_list if X_raw[c].dtype == "object" or pd.api.types.is_string_dtype(X_raw[c])]
cat_cols = sorted(set(auto_cat) | (set(raw_feat_list) & FORCE_CAT))

X_enc = pd.get_dummies(X_raw, columns=cat_cols, drop_first=True)

# Align to the exact columns the saved model was trained on (order + presence)
X_enc = X_enc.reindex(columns=THE_FEATURE_COLUMNS, fill_value=0.0)

# --- 2) Chronological split + internal validation days (same logic as before) ---
cutoff_day = df["DAGNR"].max() - 7
train_mask = df["DAGNR"] <= cutoff_day
test_mask  = df["DAGNR"] >  cutoff_day

train_days = np.sort(df.loc[train_mask, "DAGNR"].unique())
val_days_count = max(1, int(0.2 * len(train_days)))
val_days = set(train_days[-val_days_count:])
tr_days  = set(train_days[:-val_days_count]) if val_days_count < len(train_days) else set()
if len(tr_days) == 0:
    val_days = set(train_days[-2:])
    tr_days  = set(train_days[:-2])

train_sub_mask = train_mask & df["DAGNR"].isin(tr_days)
valid_mask     = train_mask & df["DAGNR"].isin(val_days)

X_tr, y_tr = X_enc[train_sub_mask], df.loc[train_sub_mask, "REALISATIE"].astype(float)
X_va, y_va = X_enc[valid_mask],     df.loc[valid_mask, "REALISATIE"].astype(float)
X_te, y_te = X_enc[test_mask],      df.loc[test_mask,  "REALISATIE"].astype(float)

# --- 3) Disrupted masks (optional reporting) ---
NO_DELAY_LABEL = "no delay (0-1 min)"
NO_DISR_LABEL  = "no disruption"
def disrupted_mask(idx):
    dcat = df.loc[idx, "delay_category"].astype(str).str.strip().str.lower() if "delay_category" in df.columns else "nan"
    disc = df.loc[idx, "disruption_category"].astype(str).str.strip().str.lower() if "disruption_category" in df.columns else "nan"
    return (dcat != NO_DELAY_LABEL) | (disc != NO_DISR_LABEL)

va_dis = disrupted_mask(df.index[valid_mask]).values
te_dis = disrupted_mask(df.index[test_mask]).values

# --- 4) Candidates (linear + regularized) ---
cands = []
cands.append(("Linear", Pipeline([("scaler", StandardScaler(with_mean=False)), ("model", LinearRegression())])))
for a in [0.1, 1.0, 3.0, 10.0, 30.0, 100.0]:
    cands.append((f"Ridge({a})", Pipeline([("scaler", StandardScaler(with_mean=False)), ("model", Ridge(alpha=a, random_state=42))])))
for a in [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1.0]:
    cands.append((f"Lasso({a})", Pipeline([("scaler", StandardScaler(with_mean=False)), ("model", Lasso(alpha=a, max_iter=20000, random_state=42))])))
for a in [1e-3, 1e-2, 1e-1, 0.5]:
    for l1 in [0.2, 0.5, 0.8]:
        cands.append((f"ElasticNet(a={a},l1={l1})",
                      Pipeline([("scaler", StandardScaler(with_mean=False)),
                                ("model", ElasticNet(alpha=a, l1_ratio=l1, max_iter=20000, random_state=42))])))

# --- 5) Optimize for overall MAE on validation ---
best_label, best_pipe, best_val = None, None, np.inf
for label, pipe in cands:
    pipe.fit(X_tr, y_tr)
    y_hat_va = pipe.predict(X_va)
    mae_va = mean_absolute_error(y_va, y_hat_va)
    print(f"{label:<26} | VAL MAE (all): {mae_va:.3f}")
    if mae_va < best_val:
        best_val, best_pipe, best_label = mae_va, pipe, label

# --- 6) Evaluate tuned vs THE (untuned) on TEST with correct aligned design matrix ---
y_hat_te_tuned = best_pipe.predict(X_te)
mae_te  = mean_absolute_error(y_te, y_hat_te_tuned)
rmse_te = np.sqrt(mean_squared_error(y_te, y_hat_te_tuned))
r2_te   = r2_score(y_te, y_hat_te_tuned)

print(f"\nBest tuned: {best_label} | VAL MAE={best_val:.3f}")
print("Best tuned — TEST (all):  MAE={:.2f} RMSE={:.2f} R²={:.3f}".format(mae_te, rmse_te, r2_te))

# THE_MODEL_PIPE was trained in Cell 1 on the same column set. Use identical X_enc columns here:
y_hat_te_the = THE_MODEL_PIPE.predict(X_enc[test_mask])  # SAME X_enc design matrix / columns
mae_the  = mean_absolute_error(y_te, y_hat_te_the)
rmse_the = np.sqrt(mean_squared_error(y_te, y_hat_te_the))
r2_the   = r2_score(y_te, y_hat_te_the)
print("THE model — TEST (all):   MAE={:.2f} RMSE={:.2f} R²={:.3f}".format(mae_the, rmse_the, r2_the))

# --- 7) Choose model to carry forward ---
if mae_te <= mae_the:
    SELECTED_MODEL_NAME = f"Tuned {best_label}"
    SELECTED_PIPE = best_pipe
else:
    SELECTED_MODEL_NAME = f"Untuned Linear ({THE_FEATURE_SET_NAME})"
    SELECTED_PIPE = THE_MODEL_PIPE

print(f"\nSelected model for further steps: {SELECTED_MODEL_NAME}")


Linear                     | VAL MAE (all): 64.038
Ridge(0.1)                 | VAL MAE (all): 64.037
Ridge(1.0)                 | VAL MAE (all): 64.037
Ridge(3.0)                 | VAL MAE (all): 64.036
Ridge(10.0)                | VAL MAE (all): 64.032
Ridge(30.0)                | VAL MAE (all): 64.022
Ridge(100.0)               | VAL MAE (all): 63.987
Lasso(0.0001)              | VAL MAE (all): 64.038
Lasso(0.001)               | VAL MAE (all): 64.037
Lasso(0.01)                | VAL MAE (all): 64.035
Lasso(0.1)                 | VAL MAE (all): 64.012
Lasso(0.5)                 | VAL MAE (all): 63.948
Lasso(1.0)                 | VAL MAE (all): 63.912
ElasticNet(a=0.001,l1=0.2) | VAL MAE (all): 64.012
ElasticNet(a=0.001,l1=0.5) | VAL MAE (all): 64.021
ElasticNet(a=0.001,l1=0.8) | VAL MAE (all): 64.031
ElasticNet(a=0.01,l1=0.2)  | VAL MAE (all): 63.807
ElasticNet(a=0.01,l1=0.5)  | VAL MAE (all): 63.885
ElasticNet(a=0.01,l1=0.8)  | VAL MAE (all): 63.972
ElasticNet(a=0.1,l1=0.2)   | VA

### Best fitting model's coefficient importance

In [28]:
# --- Standardized coefficients for BEST feature set model ---

from sklearn.preprocessing import StandardScaler

best_name = "Set 4: Base + Disruption flags and cats"
best_features = feature_sets[best_name]
best_features = [c for c in best_features if c in df.columns]
X_best = df[best_features].copy()

# one-hot encode categoricals
cat_cols_best = [c for c in best_features if X_best[c].dtype == "object" or pd.api.types.is_string_dtype(X_best[c])]
X_best = pd.get_dummies(X_best, columns=cat_cols_best, drop_first=True)

# use only training split
X_train_best = X_best[train_mask]
y_train_best = y[train_mask]

# standardize and fit linear model
scaler = StandardScaler()
X_scaled_best = scaler.fit_transform(X_train_best)
model_std = LinearRegression()
model_std.fit(X_scaled_best, y_train_best)

# collect coefficients
std_coefs = pd.DataFrame({
    "Feature": X_train_best.columns,
    "Standardized_Coefficient": model_std.coef_,
    "AbsValue": np.abs(model_std.coef_)
}).sort_values("AbsValue", ascending=False)

print(f"\n=== Top 15 Standardized Coefficients — {best_name} ===")
print(std_coefs.head(15).to_string(index=False))



=== Top 15 Standardized Coefficients — Set 4: Base + Disruption flags and cats ===
                           Feature  Standardized_Coefficient   AbsValue
                   PROGNOSE_REIZEN                203.272979 203.272979
              DAGDEELTREIN_Daluren                -18.429856  18.429856
              DAGDEELTREIN_Weekend                -18.006919  18.006919
           Previous train canceled                 17.039733  17.039733
              DAGDEELTREIN_Unknown                 12.810525  12.810525
                        ExtraTrain                 12.810525  12.810525
         DAGDEELTREIN_Ochtendspits                -11.300351  11.300351
    delay_category_Small (1-5 min)                  9.353866   9.353866
  delay_category_Medium (5-10 min)                  8.069645   8.069645
 disruption_category_No Disruption                  6.688127   6.688127
               train_type_Sprinter                  3.673553   3.673553
                    train_type_TGV                  

## MLP Model

In [47]:
# MLP on the same feature set


import tensorflow as tf
from tensorflow import keras

# --- 1) Rebuild from RAW features of THE_FEATURE_SET_NAME, re-encode, then align to THE_FEATURE_COLUMNS ---
raw_feat_list = feature_sets[THE_FEATURE_SET_NAME]
raw_feat_list = [c for c in raw_feat_list if c in df.columns]
X_raw = df[raw_feat_list].copy()

FORCE_CAT = {"DAGDEELTREIN","train_type","delay_category","disruption_category"}
auto_cat = [c for c in raw_feat_list if X_raw[c].dtype == "object" or pd.api.types.is_string_dtype(X_raw[c])]
cat_cols = sorted(set(auto_cat) | (set(raw_feat_list) & FORCE_CAT))

X_enc = pd.get_dummies(X_raw, columns=cat_cols, drop_first=True)
# align exactly to the post-one-hot training columns used by THE model
X_enc = X_enc.reindex(columns=THE_FEATURE_COLUMNS, fill_value=0.0)

# --- 2) Chronological split + internal validation inside train ---
cutoff_day = df["DAGNR"].max() - 7
train_mask = df["DAGNR"] <= cutoff_day
test_mask  = df["DAGNR"] >  cutoff_day

train_days = np.sort(df.loc[train_mask, "DAGNR"].unique())
val_days_count = max(1, int(0.2 * len(train_days)))
val_days = set(train_days[-val_days_count:])
tr_days  = set(train_days[:-val_days_count]) if val_days_count < len(train_days) else set()
if len(tr_days) == 0:
    val_days = set(train_days[-2:])
    tr_days  = set(train_days[:-2])

train_sub_mask = train_mask & df["DAGNR"].isin(tr_days)
valid_mask     = train_mask & df["DAGNR"].isin(val_days)

X_tr = X_enc[train_sub_mask].values.astype("float32")
y_tr = df.loc[train_sub_mask, "REALISATIE"].astype(float).values.astype("float32")
X_va = X_enc[valid_mask].values.astype("float32")
y_va = df.loc[valid_mask, "REALISATIE"].astype(float).values.astype("float32")
X_te = X_enc[test_mask].values.astype("float32")
y_te = df.loc[test_mask,  "REALISATIE"].astype(float).values.astype("float32")

# --- 3) Scale (safe with one-hots) ---
scaler = StandardScaler(with_mean=False)
X_tr = scaler.fit_transform(X_tr)
X_va = scaler.transform(X_va)
X_te = scaler.transform(X_te)

# --- 4) MLP ---
tf.keras.backend.clear_session()
model = keras.Sequential([
    keras.layers.Input(shape=(X_tr.shape[1],)),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)
])
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mae")
es = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor="val_loss", verbose=0)

model.fit(X_tr, y_tr, validation_data=(X_va, y_va),
          epochs=200, batch_size=1024, callbacks=[es], verbose=0)

# --- 5) Evaluate MLP vs the selected classical model on the EXACT same X_enc ---
y_pred_nn = model.predict(X_te, verbose=0).ravel()
mae_nn  = mean_absolute_error(y_te, y_pred_nn)
rmse_nn = np.sqrt(mean_squared_error(y_te, y_pred_nn))
r2_nn   = r2_score(y_te, y_pred_nn)

y_hat_sel = SELECTED_PIPE.predict(X_enc[test_mask])  # uses same aligned matrix
mae_sel  = mean_absolute_error(y_te, y_hat_sel)
rmse_sel = np.sqrt(mean_squared_error(y_te, y_hat_sel))
r2_sel   = r2_score(y_te, y_hat_sel)

print(f"\nMLP — Test: MAE={mae_nn:.2f} RMSE={rmse_nn:.2f} R²={r2_nn:.3f}")
print(f"{SELECTED_MODEL_NAME} — Test: MAE={mae_sel:.2f} RMSE={rmse_sel:.2f} R²={r2_sel:.3f}")

print("Winner:", "MLP (lower MAE)" if mae_nn < mae_sel else "Selected linear model (lower MAE)")

# --- Disrupted-only metrics (delay != 'No delay (0-1 min)' OR disruption != 'No Disruption') ---

NO_DELAY_LABEL = "no delay (0-1 min)"
NO_DISR_LABEL  = "no disruption"

def disrupted_mask(idx):
    dcat = df.loc[idx, "delay_category"].astype(str).str.strip().str.lower() if "delay_category" in df.columns else "nan"
    disc = df.loc[idx, "disruption_category"].astype(str).str.strip().str.lower() if "disruption_category" in df.columns else "nan"
    return (dcat != NO_DELAY_LABEL) | (disc != NO_DISR_LABEL)

test_idx = df.index[test_mask]
dis_mask_test = disrupted_mask(test_idx).values
n_dis = int(dis_mask_test.sum())
print(f"\nDisrupted runs in TEST (OR rule): {n_dis} / {len(dis_mask_test)}")

if n_dis > 0:
    # MLP on disrupted
    mae_nn_d  = mean_absolute_error(y_te[dis_mask_test], y_pred_nn[dis_mask_test])
    rmse_nn_d = np.sqrt(mean_squared_error(y_te[dis_mask_test], y_pred_nn[dis_mask_test]))
    r2_nn_d   = r2_score(y_te[dis_mask_test], y_pred_nn[dis_mask_test])

    # Selected linear on disrupted (uses same aligned X_enc)
    y_hat_sel_d = y_hat_sel[dis_mask_test]
    mae_sel_d  = mean_absolute_error(y_te[dis_mask_test], y_hat_sel_d)
    rmse_sel_d = np.sqrt(mean_squared_error(y_te[dis_mask_test], y_hat_sel_d))
    r2_sel_d   = r2_score(y_te[dis_mask_test], y_hat_sel_d)

    # Operator baseline on disrupted (optional)
    op_test = df.loc[test_mask, "PROGNOSE_REIZEN"].astype(float).values
    op_test_d = op_test[dis_mask_test]
    mae_op_d  = mean_absolute_error(y_te[dis_mask_test], op_test_d)
    rmse_op_d = np.sqrt(mean_squared_error(y_te[dis_mask_test], op_test_d))
    r2_op_d   = r2_score(y_te[dis_mask_test], op_test_d)

    print("\nMLP — Disrupted runs:")
    print(f"MAE={mae_nn_d:.2f} RMSE={rmse_nn_d:.2f} R²={r2_nn_d:.3f}")

    print(f"{SELECTED_MODEL_NAME} — Disrupted runs:")
    print(f"MAE={mae_sel_d:.2f} RMSE={rmse_sel_d:.2f} R²={r2_sel_d:.3f}")

    print("Operator — Disrupted runs:")
    print(f"MAE={mae_op_d:.2f} RMSE={rmse_op_d:.2f} R²={r2_op_d:.3f}")
else:
    print("No disrupted runs found in test set.")




MLP — Test: MAE=57.09 RMSE=88.71 R²=0.830
Untuned Linear (Set 4: Base + Disruptions full) — Test: MAE=59.21 RMSE=91.04 R²=0.821
Winner: MLP (lower MAE)

Disrupted runs in TEST (OR rule): 7767 / 23459

MLP — Disrupted runs:
MAE=66.39 RMSE=103.07 R²=0.806
Untuned Linear (Set 4: Base + Disruptions full) — Disrupted runs:
MAE=67.84 RMSE=104.50 R²=0.801
Operator — Disrupted runs:
MAE=71.26 RMSE=113.18 R²=0.766
